# Data Project Pitch

### Σύνοψη 

Θα πραγματοποιηθεί μια έρευνα για τις εκτάσεις που έχουν καεί στην περιοχή της Αττικής, μέσα απο τα ανοιχτά δεδομένα που υπάρχουν στον ιστότοπο του πυροσβεστικού σώματος Ελλάδος. Τα αρχεία αυτά περιλαμβάνουν κατηγορίες συνόλων δεδομένων που διατίθενται και σε ανοιχτό μηχαναγνώσιμο μορφότυπο. Θα αναλυθούν τα δεδομένα που αφορούν τα δασικά συμβάντα, στα οποία επεμβαίνει το Πυροσβεστικό Σώμα. Ο υπερσύνδεσμος, που περιλαμβάνει τα σχετικά αρχεία τύπου (xls), που σε αυτά θα γίνει scraping είναι ο ακόλουθος: https://www.fireservice.gr/el/synola-dedomenon

### Έμπνευση

- Η έμπνευση προέρχεται απο τις εμπειρίες των μελών της ομάδας, τις οποίες αποκόμισαν απο τις φωτιές στην περιοχή των Θρακομακεδόνων το καλοκαίρι του 2022.
- Θα προσπαθήσουμε να δημιουργήσουμε έναν αντίστοιχο χάρτη,
με αυτόν του υπερσυνδέσμου του Υπουργείου Πολιτικής Προστασίας, 
που θα απεικονίζει την συχνότητα των περιοχών που έχουν καεί και όχι τον κίνδυνο φωτιάς:
https://www.civilprotection.gr/el/%CE%B7%CE%BC%CE%B5%CF%81%CE%AE%CF%83%CE%B9%CE%BF%CF%82-%CF%87%CE%AC%CF%81%CF%84%CE%B7%CF%82-%CF%80%CF%81%CF%8C%CE%B2%CE%BB%CE%B5%CF%88%CE%B7%CF%82-%CE%BA%CE%B9%CE%BD%CE%B4%CF%8D%CE%BD%CE%BF%CF%85-%CF%80%CF%85%CF%81%CE%BA%CE%B1%CE%B3%CE%B9%CE%AC%CF%82-01082008
Διαφορετικά θα θέλαμε να κάνουμε κάτι πιο σύνθετο όπως land.copernicus:
https://land.copernicus.eu/pan-european/corine-land-cover/clc-2006/#

### Περαιτέρω πληροφορίες για το θέμα σας

- <b>Τίτλος εργασίας</b>:<br>
Fire data in Attica since 2000<br>
Μελέτη περίπτωσης όσον αφορά τις δασικές πυρκαγίες στην Αττική απο το 2000 έως το 2021 με παραμέτρους μελέτης 
<br><br>
- <b>Δεδομένα</b>: <i> https://www.fireservice.gr/el/synola-dedomenon</i>
<br><br>
- <b>Πιθανά προβλήματα/ανασφάλειες/απορίες</b>:

## Ερευνητικά ερωτήματα

###Τα ερευνητικά ερωτήματα που υπάρχουν είναι τα εξής:<br>

####1)	 Συχνότητα που καιγονται περιοχές (πόσες φορές έχουν καεί συγκεκριμένες περιοχές) <br>


#####Εισαγωγή βιβλιοθηκών

In [65]:
import requests 
from bs4 import BeautifulSoup
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
from urllib.request import urlopen, urlretrieve
import pandas as pd
import re 
import time
import pandas as pd
import numpy as np

#####Κατέβασμα αρχείων δεδομένων

In [2]:
disable_warnings(InsecureRequestWarning) # disabling warnings because the site is overly secured

page = requests.get('https://www.fireservice.gr/el/synola-dedomenon', verify=False) # verify = False (not using SSL verification)

html = BeautifulSoup(page.text, 'html.parser')

In [3]:
pattern = re.compile('Δασικές Πυρκαγιές \d{4}-?\d?\d?\d?\d?') # patterns of the links we want
pattern2 = re.compile('\d{4}-?\d?\d?\d?\d?')
spans = html.find_all('span') # finding all the spans in the html file 
texts = [i.text for i in spans] # getting the texts from all the spans

idxs = []
for i in range(len(texts)): # going through the indexes of the text
  if re.match(pattern, texts[i]):
    idxs.append(i) # adding the indexes that correspond to the spans we want 

# getting the spans with the links we want 
wanted_spans = []
for i in idxs:
  wanted_spans.append(spans[i])

wanted_texts = [i.text for i in wanted_spans] # text from wanted_spans
filenames = [] # contains the names of the files 
for i in wanted_texts:
  matches = pattern2.finditer(i)
  for j in matches:
    filenames.append(j.group())
filenames = list(dict.fromkeys(filenames)) # removing duplicates

xlsx_urls = []
# getting the urls of xlsx files 
for span in wanted_spans:
  xlsx_urls.append(span.find_all('a')[1]['href'])

xlsx_urls = list(dict.fromkeys(xlsx_urls)) # removing duplicates
base_string = 'https://www.fireservice.gr/' # the urls lack the base string 

# adding the base string 
for i in range(len(xlsx_urls)):
  xlsx_urls[i] = 'https://www.fireservice.gr/' + xlsx_urls[i]


c = 0
for doc in xlsx_urls:
  time.sleep(2)
  response = requests.get(doc, allow_redirects=True, verify=False)
  f = open(str(filenames[c])+".xlsx", 'wb')
  c += 1
  f.write(response.content)
  f.close()


#####Εξερεύνηση δεδομένων

In [4]:
# renaming the columns 
def col_rename(df):
  new = list(df.iloc[0,:].to_numpy())
  old = list(df.columns)

  change = {o : n for (o, n) in zip(old, new)}
  df = df.rename( columns = change)
  df = df.iloc[1:,:]
  return df



In [5]:
# initializing dataframes
df_00_12 = pd.read_excel('2000-2012.xlsx')
df_13 = pd.read_excel('2013.xlsx')
df_14 = pd.read_excel('2014.xlsx')
df_15 = pd.read_excel('2015.xlsx')
df_16 = pd.read_excel('2016.xlsx')
df_17 = pd.read_excel('2017.xlsx')
df_18 = pd.read_excel('2018.xlsx')
df_19 = pd.read_excel('2019.xlsx')
df_20 = pd.read_excel('2020.xlsx')
df_21 = pd.read_excel('2021.xlsx')


In [6]:
#changing the column names of some problematic datasets
df_00_12 = col_rename(df_00_12)
df_13 = col_rename(df_13)
df_15 = col_rename(df_15)
df_16 = col_rename(df_16)
df_17 = col_rename(df_17)
df_18 = col_rename(df_18)
df_19 = col_rename(df_19)
df_20 = col_rename(df_20)
df_21 = col_rename(df_21)


In [7]:
df_00_12.rename( columns = {'Περιοχή - Τοποθεσία' : 'Περιοχή'}, inplace=True)

In [8]:
# Gathering all dataframes into one
df = df_00_12.append(
    df_13.append(
    df_14.append(
    df_15.append(
    df_16.append(
    df_17.append(
    df_18.append(
    df_19.append(
    df_20.append(
    df_21
    )))))))))

# filtering the dataframes to return only information on Attica
df_attiki = df[df['Νομός'] == 'ΑΤΤΙΚΗΣ'] 
# removing dupplicates based on date so that we don't add more than they truly are 
df_attiki_unique = df_attiki.drop_duplicates(subset=['Ημερ/νία Έναρξης'])
# keeping only the location feature and dropping all nan values
location = df_attiki_unique['Περιοχή'].dropna()


In [9]:
fire_counts = dict(location.value_counts())
print('Συχνότητες καμένων περιοχών κατά την περίοδο 2000-2021: \n')
for i in fire_counts.keys():
  if i != '-':
    print(i, ':', fire_counts[i])


Συχνότητες καμένων περιοχών κατά την περίοδο 2000-2021: 

ΚΕΡΑΤΕΑ : 142
ΜΕΓΑΡΑ : 41
ΑΝΑΒΥΣΣΟΣ : 21
ΛΑΥΡΙΟ : 18
ΑΧΑΡΝΑΙ : 17
ΚΑΠΑΝΔΡΙΤΙ : 16
ΚΑΛΑΜΟΣ : 15
ΠΑΛΙΑ ΦΩΚΑΙΑ : 12
ΒΙΛΙΑ : 11
ΚΡΥΟΝΕΡΙ : 9
ΑΦΙΔΝΕΣ : 8
ΑΣΠΡΟΠΥΡΓΟΣ : 8
ΣΤΑΜΑΤΑ : 8
ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ : 7
ΠΕΤΡΟΥΠΟΛΗ : 6
ΑΝΩ ΛΙΟΣΙΑ : 6
ΦΥΛΗ : 6
ΚΑΜΑΤΕΡΟΥ : 6
ΣΑΡΩΝΙΔΑ : 5
ΜΑΡΚΟΠΟΥΛΟ : 5
ΘΡΑΚΟΜΑΚΕΔΟΝΕΣ : 5
ΚΗΦΙΣΙΑ : 4
ΒΑΣΙΛΙΚΑ : 3
ΜΑΓΟΥΛΑ : 3
ΚΟΥΛΟΥΡΙΩΤΙΚΟ ΜΟΝΟΠΑΤΙ : 3
ΠΕΡΑΜΑ : 3
ΠΕΝΤΕΛΗ : 3
ΦΑΝΕΡΩΜΕΝΗ : 3
ΟΙΝΟΗ : 3
ΜΗΤΑΤΑ : 3
ΠΟΛΥΔΕΝΔΡΙ : 3
ΙΚΑΡΙΩΤΙΚΑ ΠΕΡΑΜΑΤΟΣ : 2
ΠΙΚΕΡΜΙ : 2
ΚΟΥΒΑΡΑΣ : 2
ΧΑΙΔΑΡΙ : 2
ΑΓ.ΛΟΥΚΑΣ : 2
ΣΑΛΑΜΙΝΑ : 2
ΛΕΩΦ.ΝΑΤΟ ΑΣΠΡΟΠΥΡΓΟΥ : 2
ΜΑΝΔΡΑ : 2
ΜΕΘΑΝΑ : 2
ΜΑΡΟΥΔΙ ΑΙΑΣ ΚΛΑΜΠ : 1
ΑΓ. ΓΕΩΡΓΙΟΣ ΤΡΟΙΖΗΝΙΑΣ : 1
ΓΑΛΑΤΑΣ-ΔΑΡΙΖΑ : 1
ΔΗΜΟΣ ΤΡΟΙΖΗΝΑΣ : 1
ΓΑΛΑΤΑΣ-ΚΟΚΚΙΝΙΑ ΤΡΟΙΖΗΝΑΣ : 1
ΚΑΤΑΣΚΗΝΩΣΗ ΓΑΛΑΤΑ ΤΡΟΙΖΗΝΑ : 1
ΜΑΡΚΟΠΟΥΛΟ ΩΡΩΠΟΥ : 1
ΤΟΡΟΝΤΟ ΑΜΠΕΛΑΚΙΩΝ : 1
ΠΕΡΙΣΤΕΡΙΑ : 1
ΛΕΩΦ.ΦΑΝΕΡΩΜΕΝΗΣ ΑΓ.ΓΕΩΡΓΙΟΣ : 1
ΑΙΑΝΤΕΙΟ : 1
ΑΓΙΟΣ ΑΝΤΩΝΙΟΣ ΑΜΠΕΛΑΚΙΩΝ : 1
ΘΕΣΗ ΣΤΡΩΜΑ ΥΜΗΤΤΟΥ ΔΑΣΟΜΕΝΗ ΚΟΡΥΦΗ : 1
ΑΜΦΙΑΡΕΙΟ ΚΑΛΑΜΟΥ ΑΤΤΙΚΗΣ : 1
ΠΑΙΑΝΙΑ 

In [68]:
loc_counts_df = pd.DataFrame(location.value_counts())
loc_counts_df.columns = [''] * len(loc_counts_df.columns)
loc_counts_df = loc_counts_df[np.invert(loc_counts_df.index == '-')]
loc_counts_df.to_excel('fire_counts.xlsx')

In [10]:
# saving the fires with their unique values for the creation of the map 
df_map = df_attiki_unique.drop_duplicates(subset=['Περιοχή'])

# normalizing fire_counts so that the counts represent probability weights
fire_counts_normalized = {i : j for i,j in zip(fire_counts.keys() , list(list(fire_counts.values()) / max(list(fire_counts.values()))))}
df_map.index = range(191)
# adding the weights to the dataframe 
for i in list(fire_counts_normalized.keys()):
  for j in range(len(df_map)):
    if i == df_map.loc[j, "Περιοχή"]:
      df_map.loc[j, 'intensity'] = fire_counts_normalized[i]
      
df_map.to_excel('fires.xlsx')


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
print('Περιοχή με τις περισσότερες φωτιές στο διάστημα της περιόδου 2000-2021 = {}({})'
      .format(max(fire_counts, key=lambda key: fire_counts[key]),  max(list(fire_counts.values())) ))

Περιοχή με τις περισσότερες φωτιές στο διάστημα της περιόδου 2000-2021 = ΚΕΡΑΤΕΑ(142)


####2)	Σύνολο καμμένων εκτάσεων <br>


In [285]:
len(fire_counts)

190

####3)	 Πόσες δασικές και πόσες γεωργικές εκτάσεις <br>


In [280]:
ektaseis = df[['Γεωργικές Εκτάσεις', 'Δασική Έκταση']]
ektaseis.to_excel('ektaseis.xlsx')
ektaseis

Γεωργικές Εκτάσεις Δασική Έκταση
1                     0             3
2                     8             0
3                     0             2
4                     0             2
5                     0             0
...                 ...           ...
9510               0.05             0
9511                  0             1
9512                  0             1
9513                  0          0.05
9514                  0             0

[97562 rows x 2 columns]

In [286]:
ektaseis.sum()

Γεωργικές Εκτάσεις     843143.57
Δασική Έκταση         1443661.11
dtype: object

####4)	Χρόνος κατάσβεσης<br>

In [233]:
fire_time = df[['Περιοχή','Ημερ/νία Έναρξης', 'Ημερ/νία Κατασβεσης', 'Ώρα Έναρξης', 'Ώρα Κατάσβεσης']].dropna()

# cleaning problematic data
drop_list = []
p = re.compile(r'^\d{4}-\d{2}-\d{2}')
for i in range(len(fire_time.iloc[:,2])):
  match = re.match(p, str(fire_time.iloc[i,2]))
  if match == None:
    drop_list.append(i)
fire_time = fire_time.drop(index=drop_list)


In [234]:
# correcting the start time 
for i in range(len(fire_time.iloc[:,3])):
  fire_time.iloc[i, 3] = pd.to_datetime(str(fire_time.iloc[i, 3])).time()


In [235]:
for i in range(len(fire_time.iloc[:,4])):
  fire_time.iloc[i, 4] = pd.to_datetime(str(fire_time.iloc[i, 4])).time()


In [236]:
# correcting the start dates
for i in range(len(fire_time.iloc[:,1])):
  fire_time.iloc[i, 1] = pd.to_datetime(str(fire_time.iloc[i, 1])).date()



In [237]:
for i in range(len(fire_time.iloc[:,2])):
  try:
    fire_time.iloc[i, 2] = pd.to_datetime(str(fire_time.iloc[i, 2])).date()
  except:
    try:
      fire_time = fire_time.drop(index=[i])
    except:
      continue


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [265]:
for i in range(len(fire_time)):
  try:
    # day difference in hours
    daydiff = (pd.to_datetime(str(fire_time.iloc[i,2])) - pd.to_datetime(str(fire_time.iloc[i,1]))).total_seconds() / 3600
    timediff = (pd.to_datetime(str(fire_time.iloc[i,4])) - pd.to_datetime(str(fire_time.iloc[i,3]))).total_seconds() / 3600
  
    # adding the time to the df
    fire_time.loc[i,'Χρόνος κατάσβεσης(ώρες)'] = round(daydiff + timediff, 2)

  except:
      continue



In [277]:
fire_time = fire_time.dropna()

In [279]:
fire_time

Περιοχή Ημερ/νία Έναρξης Ημερ/νία Κατασβεσης  \
1                    ΚΡΥΟΝΕΡΙ       2000-06-08          2000-06-08   
2             ΑΓΡ.ΠΕΡ.ΑΦΙΔΝΩΝ       2000-06-05          2000-06-05   
3                      ΚΑΡΕΑΣ       2000-06-14          2000-06-14   
4     ΝΕΚΡΟΤΑΦΕΙΟ ΚΑΙΣΑΡΙΑΝΗΣ       2000-07-23          2000-07-23   
5      ΔΡΟΣΟΠΟΥΛΟΥ ΑΝΩ ΛΙΟΣΙΑ       2000-10-04          2000-10-04   
...                       ...              ...                 ...   
9394                        -       2021-10-01          2021-10-03   
9449                        -       2021-11-06          2021-11-06   
9459                   ΑΓΑΛΑΣ       2021-02-23          2021-02-23   
9470                   ΑΓΑΛΑΣ       2021-04-09          2021-04-09   
9491                  ΜΟΥΖΑΚΙ       2021-08-03          2021-08-03   

     Ώρα Έναρξης Ώρα Κατάσβεσης  Χρόνος κατάσβεσης(ώρες)  
1       13:30:00       15:00:00                     1.27  
2       11:44:00       13:00:00                     3.58  
3       10:25:00       14:00:00                     2.10  
4       11:10:00       13:16:00                     2.67  
5       14:40:00       17:20:00                     4.50  
...          ...            ...                      ...  
9394    22:31:00       21:00:00                    11.77  
9449    18:15:00       20:12:00                     0.22  
9459    13:38:00       15:30:00                     0.68  
9470    16:27:00       19:57:00                     2.52  
9491    13:45:00       17:00:00                     1.43  

[37138 rows x 6 columns]

In [283]:
fire_time.to_excel('fire_time.xlsx')

In [287]:
fire_time

Περιοχή Ημερ/νία Έναρξης Ημερ/νία Κατασβεσης  \
1                    ΚΡΥΟΝΕΡΙ       2000-06-08          2000-06-08   
2             ΑΓΡ.ΠΕΡ.ΑΦΙΔΝΩΝ       2000-06-05          2000-06-05   
3                      ΚΑΡΕΑΣ       2000-06-14          2000-06-14   
4     ΝΕΚΡΟΤΑΦΕΙΟ ΚΑΙΣΑΡΙΑΝΗΣ       2000-07-23          2000-07-23   
5      ΔΡΟΣΟΠΟΥΛΟΥ ΑΝΩ ΛΙΟΣΙΑ       2000-10-04          2000-10-04   
...                       ...              ...                 ...   
9394                        -       2021-10-01          2021-10-03   
9449                        -       2021-11-06          2021-11-06   
9459                   ΑΓΑΛΑΣ       2021-02-23          2021-02-23   
9470                   ΑΓΑΛΑΣ       2021-04-09          2021-04-09   
9491                  ΜΟΥΖΑΚΙ       2021-08-03          2021-08-03   

     Ώρα Έναρξης Ώρα Κατάσβεσης  Χρόνος κατάσβεσης(ώρες)  
1       13:30:00       15:00:00                     1.27  
2       11:44:00       13:00:00                     3.58  
3       10:25:00       14:00:00                     2.10  
4       11:10:00       13:16:00                     2.67  
5       14:40:00       17:20:00                     4.50  
...          ...            ...                      ...  
9394    22:31:00       21:00:00                    11.77  
9449    18:15:00       20:12:00                     0.22  
9459    13:38:00       15:30:00                     0.68  
9470    16:27:00       19:57:00                     2.52  
9491    13:45:00       17:00:00                     1.43  

[37138 rows x 6 columns]

## Προέρευνα/δουλειά που έχετε ήδη κάνει στο πλαίσιο του σχεδιασμού του πρότζεκτ

Η προεργασία, που κάναμε αφορούσε την έρευση ενός θέματος για το οποίο υπήρχαν διαθέσιμες κατηγορίες συνόλων δεδομένων 
που διατίθενται και σε ανοιχτό μηχαναγνώσιμο μορφότυπο. Καταλήξαμε στην επιλογή του θέματος με τις πυρκαγίες, καθώς στον ιστότοπο του ΠΣ Ελλάδας, υπήρχαν τα δεδομένα αυτά. Επιλέξαμε στην συνέχεια τρια με τέσσερα ερευνητικά, που θέλουμε να απαντήσουμε μέσω scraping των αρχείων  xls. Τέλος φτιάξαμε το αρχείο αυτό, που μας ζητήθηκε.

## Απαντήστε "YES" ή "NO" στα παρακάτω ερωτήματα
<i>(Στο στάδιο της πρότασης θέματος που βρίσκεστε, το αναμενόμενο είναι YES. Όσα περισσότερα NO, τόσο λιγότερο έχετε προετοιμαστεί και, άρα, χρειάζεται να σχεδιάσετε καλύτερα το πρότζεκτ σας.)</i>

In [12]:
input("Έχετε ψάξει για πηγές δεδομένων για το θέμα που θέλετε να κάνετε;")   

In [13]:
input("Έχετε βρει δεδομένα/πηγές δεδομένων για το θέμα που θέλετε να κάνετε;" "YES")

In [14]:
input("Έχετε περάσει χρόνο εξερευνώντας τα δεδομένα σας; Έχετε καταλάβει τι έχετε, σε ποια μορφή είναι και ενδεχομένως τι σας λείπει;" "YES")

In [15]:
input("Έχετε διατυπώσει τα ερευνητικά ερωτήματά σας;" "YES")

In [16]:
input("Έχετε σκεφτεί πώς θα αναλύσετε τα δεδομένα σας, για να απαντήσετε τα ερευνητικά ερωτήματά σας;" "YES")

In [17]:
input("Έχετε αρχίσει να σκέφτεστε πώς θα μπορούσατε να οπτικοποιήσετε τις αναλύσεις σας για το τελικό σας story;" "YES")

In [18]:
input("Έχετε καταθέσει το pitch σας (το προτεινόμενο θέμα σας) σε ένα notebook με filename PITCH_ΟΝΟΜΑΤΕΠΩΝΥΜΟ1_ΟΝΟΜΑΤΕΠΩΝΥΜΟ2;" "ΑΥΤΟ ΕΔΩ ΕΙΝΑΙ")

### Feedback!
Mπράβο για την ιδέα και που έχετε ήδη αρχίσει να σχεδιάζετε το πρότζεκτ σας! 

Προτείνω τα εξής: 
1. Διευκρινίστε εάν θα ασχοληθείτε μόνο με δασικά συμβάντα ή και με αστικά
2. Διευκρινίστε ποιο είναι το χρονικό διάστημα μελέτης
3. Περάστε χρόνο με τα δεδομένα σας: εξερευνήστε τα data για το χρονικό διάστημα που σας ενδιαφέρει και καταλάβετε τι έχετε, τι δεν έχετε και ποιες τυχόν διαφορές υπάρχουν από χρονιά σε χρονιά στα data που παρέχονται
4. Σκεφτείτε τι ακριβώς θα χαρτογραφήσετε: δήμους; περιοχές; ακριβή τοποθεσία συμβάντος; 
5. Με βάση το 4, αρχίστε να σκέφτεστε μια στρατηγική για το πώς θα χειριστείτε τα δεδομένα σας, ώστε να πετύχετε τη χαρτογράφηση που θέλετε

Όσον αφορά στη συλλογή δεδομένων, γράφετε: «Ο υπερσύνδεσμος, που περιλαμβάνει τα σχετικά αρχεία τύπου (xls), που σε αυτά θα γίνει scraping είναι ο ακόλουθος: https://www.fireservice.gr/el/synola-dedomenon». Δεν  χρειάζεται να κάνετε scraping, καθώς τα δεδομένα σας διατίθενται ήδη σε μηχαναγνώσιμο format. Ωστόσο, σας προτείνω να γράψετε έναν μικρό scraper που θα κάνει download τα αρχεία .xls για εξάσκηση :)

Ως προς την προεργασία/προέρευνα: Στο κομμάτι αυτό του pitch, ο σκοπός δεν είναι να κάνουμε μία περίληψη όσων έχουμε ήδη περιγράψει παραπάνω. Σκοπός της προέρευνας είναι να κάνετε μια πρώτη, πολύ πρώιμη ανάλυση που θα σας δώσει ένα πρώτο εύρημα, το οποίο θα χρησιμοποιήσετε για να εξηγήσετε σε κάποιον γιατί αυτό το κομμάτι είναι σημαντικό και γιατί έχει νόημα η περαιτέρω δημοσιογραφική έρευνα.

In [19]:
c = df_map[df_map['Περιοχή'] == 'ΚΕΡΑΤΕΑ']
c['X-ENGAGE']

146    NaN
Name: X-ENGAGE, dtype: object

In [122]:
l = pd.DataFrame(np.array([[1,2,3],[4,5,6],[7,8,9]]))

In [125]:
l.iloc[0,0] = 20

In [126]:
l

0  1  2
0  20  2  3
1   4  5  6
2   7  8  9